In [1]:
import pandas as pd
import json
import requests
import re
from datetime import datetime
import geopy
import sys
import os
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
from datetime import datetime

In [43]:
import NWSPPFunctions as parsfunc
import forecast_merger as merg
import nws

In [3]:
apistarturl = 'https://api.weather.gov'

In [4]:
PFM = parsfunc.get_api_products(29.9511,-90.0715)

{'productCode': 'PFM', 'productName': 'Point Forecast Matrices'}


In [5]:
product_json = parsfunc.returnthree_hr_timeseries_json_from_pfm(PFM)

In [6]:
product_json

[{'location': '31.18N  90.47W E',
  'start_date': 'Tue 07/26/22 ',
  'pop12H': ['50', '10', '30', ' 5', '30'],
  'QPF12H': ['  0.01', '     0', '  0.01', '     0', '  0.01'],
  'Snow12H': ['00-00', '00-00', '00-00'],
  'timeseries': [[4, '   ', '   ', '   ', '   ', '   ', '   '],
   [7, ' 73', '72 ', '97 ', ' S ', ' 2 ', 'SC '],
   [10, ' 83', '74 ', '74 ', 'SW ', ' 6 ', 'SC '],
   [13, ' 89', '73 ', '59 ', ' S ', ' 8 ', 'SC '],
   [16, ' 88', '73 ', '61 ', ' S ', ' 8 ', 'B1 '],
   [19, ' 83', '73 ', '72 ', ' S ', ' 8 ', 'SC '],
   [22, ' 77', '73 ', '87 ', ' S ', ' 4 ', 'FW '],
   [25, ' 75', '73 ', '94 ', ' S ', ' 3 ', 'FW '],
   [28, ' 73', '72 ', '97 ', ' S ', ' 2 ', 'SC '],
   [31, ' 73', '72 ', '97 ', 'SE ', ' 2 ', 'SC '],
   [34, ' 84', '74 ', '72 ', ' S ', ' 6 ', 'SC '],
   [37, ' 90', '73 ', '58 ', ' S ', ' 6 ', 'FW '],
   [40, ' 91', '72 ', '54 ', ' S ', ' 8 ', 'FW '],
   [43, ' 85', '73 ', '67 ', ' S ', ' 6 ', 'SC '],
   [46, ' 78', '73 ', '85 ', ' S ', ' 4 ', 'FW '],
   [49

In [12]:
dfpfm = pd.DataFrame(product_json)
cor = dfpfm['location'].tolist()
coords = []

for x in range(0,len(cor)):
    c1 = cor[x].replace('N ',',')
    c2 = c1.replace('W E', '')
    c3 = c2.replace(' ','-')
    coords.append(c3)
    
#In this part we give the coordinates the correct format lat, lon to make the API request of the GRID forecasts

df = pd.DataFrame(coords, columns = ['Coordinates'])
df[['lat','lon']]=df.Coordinates.str.split(',',expand=True)
del df['Coordinates']
coords = df.to_dict('index')
print(coords)

{0: {'lat': '31.18', 'lon': '-90.47'}, 1: {'lat': '30.53', 'lon': '-91.15'}, 2: {'lat': '30.25', 'lon': '-89.77'}, 3: {'lat': '30.00', 'lon': '-90.25'}, 4: {'lat': '30.07', 'lon': '-89.93'}, 5: {'lat': '30.42', 'lon': '-89.08'}, 6: {'lat': '30.47', 'lon': '-88.53'}}


In [48]:
json_forecast = []

for i in range(0,len(coords)):
    while True:
        try:
            lat = coords[i]['lat']
            lon = coords[i]['lon']
            nwsforecast = nws.collect_NWS_gridpoint_forecast_from_point(float(lat), float(lon), gridpoints_d=False)
            json_forecast.append(nwsforecast)
            
        except nwsforecast == None:
            
            lat = coords[i]['lat']
            lon = coords[i]['lon']
            nwsforecast = nws.collect_NWS_gridpoint_forecast_from_point(float(lat), float(lon), gridpoints_d=False)
            json_forecast.append(nwsforecast)
            
            continue
        break

json_forecast

{'type': 'Polygon', 'coordinates': [[[-90.4467206, 31.2172522], [-90.44761190000001, 31.1945594], [-90.4210723, 31.1937946], [-90.42017580000001, 31.2164872], [-90.4467206, 31.2172522]]]}
bad grid: 51/142
{'type': 'Polygon', 'coordinates': [[[-90.4741402, 31.1953151], [-90.4750256, 31.172629999999998], [-90.44851469999999, 31.1718704], [-90.44762409999998, 31.1945554], [-90.4741402, 31.1953151]]]}
good grid: 50/141
{'type': 'Polygon', 'coordinates': [[[-91.1328192, 30.5537028], [-91.1335721, 30.5309761], [-91.1071947, 30.5303257], [-91.10643669999999, 30.5530524], [-91.1328192, 30.5537028]]]}
bad grid: 26/112
{'type': 'Polygon', 'coordinates': [[[-91.1599535, 30.5316174], [-91.1607007, 30.5088989], [-91.13432080000001, 30.5082529], [-91.13356860000002, 30.530971299999997], [-91.1599535, 30.5316174]]]}
good grid: 25/111
{'type': 'Polygon', 'coordinates': [[[-89.7451279, 30.2859353], [-89.7461485, 30.2632044], [-89.7198465, 30.262321399999998], [-89.7188208, 30.285052099999998], [-89.745

[{'@context': ['https://geojson.org/geojson-ld/geojson-context.jsonld',
   {'@version': '1.1',
    'wmoUnit': 'https://codes.wmo.int/common/unit/',
    'nwsUnit': 'https://api.weather.gov/ontology/unit/'}],
  'id': 'https://api.weather.gov/gridpoints/LIX/50,141',
  'type': 'Feature',
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-90.4741402, 31.1953151],
     [-90.4750256, 31.172629999999998],
     [-90.44851469999999, 31.1718704],
     [-90.44762409999998, 31.1945554],
     [-90.4741402, 31.1953151]]]},
  'properties': {'@id': 'https://api.weather.gov/gridpoints/LIX/50,141',
   '@type': 'wx:Gridpoint',
   'updateTime': '2022-07-26T22:25:51+00:00',
   'validTimes': '2022-07-26T16:00:00+00:00/P8DT6H',
   'elevation': {'unitCode': 'wmoUnit:m', 'value': 136.8552},
   'forecastOffice': 'https://api.weather.gov/offices/LIX',
   'gridId': 'LIX',
   'gridX': '50',
   'gridY': '141',
   'temperature': {'uom': 'wmoUnit:degC',
    'values': [{'validTime': '2022-07-26T16:00:00+00:00/PT1H

In [80]:
pd.options.display.max_columns = None
genforecast = pd.json_normalize(json_forecast, max_level = 4)

genforecast

,@context,id,type,geometry.type,geometry.coordinates,properties.@id,properties.@type,properties.updateTime,properties.validTimes,properties.elevation.unitCode,properties.elevation.value,properties.forecastOffice,properties.gridId,properties.gridX,properties.gridY,properties.temperature.uom,properties.temperature.values,properties.dewpoint.uom,properties.dewpoint.values,properties.maxTemperature.uom,properties.maxTemperature.values,properties.minTemperature.uom,properties.minTemperature.values,properties.relativeHumidity.uom,properties.relativeHumidity.values,properties.apparentTemperature.uom,properties.apparentTemperature.values,properties.heatIndex.uom,properties.heatIndex.values,properties.windChill.uom,properties.windChill.values,properties.skyCover.uom,properties.skyCover.values,properties.windDirection.uom,properties.windDirection.values,properties.windSpeed.uom,properties.windSpeed.values,properties.windGust.uom,properties.windGust.values,properties.weather.values,properties.hazards.values,properties.probabilityOfPrecipitation.uom,properties.probabilityOfPrecipitation.values,properties.quantitativePrecipitation.uom,properties.quantitativePrecipitation.values,properties.iceAccumulation.uom,properties.iceAccumulation.values,properties.snowfallAmount.uom,properties.snowfallAmount.values,properties.snowLevel.values,properties.ceilingHeight.uom,properties.ceilingHeight.values,properties.visibility.uom,properties.visibility.values,properties.transportWindSpeed.uom,properties.transportWindSpeed.values,properties.transportWindDirection.uom,properties.transportWindDirection.values,properties.mixingHeight.uom,properties.mixingHeight.values,properties.hainesIndex.values,properties.lightningActivityLevel.values,properties.twentyFootWindSpeed.uom,properties.twentyFootWindSpeed.values,properties.twentyFootWindDirection.uom,properties.twentyFootWindDirection.values,properties.waveHeight.uom,properties.waveHeight.values,properties.wavePeriod.uom,properties.wavePeriod.values,properties.waveDirection.values,properties.primarySwellHeight.values,properties.primarySwellDirection.values,properties.secondarySwellHeight.values,properties.secondarySwellDirection.values,properties.wavePeriod2.values,properties.windWaveHeight.values,properties.dispersionIndex.values,properties.pressure.values,properties.probabilityOfTropicalStormWinds.values,properties.probabilityOfHurricaneWinds.values,properties.potentialOf15mphWinds.values,properties.potentialOf25mphWinds.values,properties.potentialOf35mphWinds.values,properties.potentialOf45mphWinds.values,properties.potentialOf20mphWindGusts.values,properties.potentialOf30mphWindGusts.values,properties.potentialOf40mphWindGusts.values,properties.potentialOf50mphWindGusts.values,properties.potentialOf60mphWindGusts.values,properties.grasslandFireDangerIndex.values,properties.probabilityOfThunder.values,properties.davisStabilityIndex.values,properties.atmosphericDispersionIndex.values,properties.lowVisibilityOccurrenceRiskIndex.values,properties.stability.values,properties.redFlagThreatIndex.values
0,[https://geojson.org/geojson-ld/geojson-contex...,"https://api.weather.gov/gridpoints/LIX/50,141",Feature,Polygon,"[[[-90.4741402, 31.1953151], [-90.4750256, 31....","https://api.weather.gov/gridpoints/LIX/50,141",wx:Gridpoint,2022-07-26T22:25:51+00:00,2022-07-26T16:00:00+00:00/P8DT6H,wmoUnit:m,136.8552,https://api.weather.gov/offices/LIX,LIX,50,141,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT2H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT10...,wmoUnit:degC,[{'validTime': '2022-07-27T01:00:00+00:00/PT14...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/P8DT...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wm

In [84]:
pd.options.display.max_rows = None
forecast = genforecast[['id', 'geometry.type','geometry.coordinates','properties.temperature.uom','properties.temperature.values','properties.dewpoint.uom','properties.dewpoint.values','properties.relativeHumidity.uom','properties.relativeHumidity.values','properties.windSpeed.uom','properties.windSpeed.values']]
forecast.explode('properties.temperature.values')
forecast

,id,geometry.type,geometry.coordinates,properties.temperature.uom,properties.temperature.values,properties.dewpoint.uom,properties.dewpoint.values,properties.relativeHumidity.uom,properties.relativeHumidity.values,properties.windSpeed.uom,properties.windSpeed.values
0,"https://api.weather.gov/gridpoints/LIX/50,141",Polygon,"[[[-90.4741402, 31.1953151], [-90.4750256, 31....",wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT2H...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...
1,"https://api.weather.gov/gridpoints/LIX/25,111",Polygon,"[[[-91.1599535, 30.5316174], [-91.1607007, 30....",wmoUnit:degC,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...,wmoUnit:percent,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...
2,"https://api.weather.gov/gridpoints/LIX/78,101",Polygon,"[[[-89.7724548, 30.2640782], [-89.7734695, 30....",wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...
3,"https://api.weather.gov/gridpoints/LIX/60,89",Polygon,"[[[-90.2571621, 30.0063661], [-90.2580811, 29....",wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...
4,"https://api.weather.gov/gridpoints/LIX/72,93",Polygon,"[[[-89.9381828, 30.0874333], [-89.939164, 30.0...",wmoUnit:degC,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...,wmoUnit:percent,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T15:00:00+00:00/PT1H...
5,"https://api.weather.gov/gridpoints/LIX/103,109",Polygon,"[[[-89.1056655, 30.4225364], [-89.1068114, 30....",wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT5H...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...
6,"https://api.weather.gov/gridpoints/LIX/124,113",Polygon,"[[[-88.5475913, 30.4916451], [-88.548846, 30.4...",wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:degC,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:percent,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...,wmoUnit:km_h-1,[{'validTime': '2022-07-26T16:00:00+00:00/PT1H...


In [ ]:
nws.adjust_compressed_nws_timeseries_to_aligned_hourly_timestamped_timeseries